In [4]:
!pip install scikit-learn

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , confusion_matrix , ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split , KFold
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score , confusion_matrix , ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split , KFold
from xgboost import XGBClassifier
import pandas as pd
from sklearn.preprocessing import LabelEncoder

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%cd '/content/drive/MyDrive/udea-IA-20251'

/content/drive/MyDrive/udea-IA-20251


In [8]:
def convertir_si_no(valor):
    if isinstance(valor, str):  # Verifica si el valor es una cadena
        if valor.lower() == 'si' or valor.lower() == 's':
            return 1
        elif valor.lower() == 'no' or valor.lower() == 'n':
            return 0
    return valor  # Devuelve el valor original si no es 'si' ni 'no' o si no es una cadena

In [9]:
def preproccesing(dir, test, size=0):

  data = pd.read_csv(dir)
  print(data.shape)
  columnas_string = data.select_dtypes(include='object').columns
  data[columnas_string] = data[columnas_string].fillna('0')
  label_encoders = {}
  for column in columnas_string:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column])
  columnas_float = data.select_dtypes(include='float64').columns
  data[columnas_float] = data[columnas_float].fillna(0)
  data[columnas_float] = data[columnas_float].astype(int)
  data = data.apply(lambda x: x.map(convertir_si_no))


  if size != 0:
    data = data.sample(n=size, random_state=42)
  else:
    pass

  if test== False:

    X = data.iloc[:,:11]
    Y = data.iloc[:,11]
    columnas_cualitativas = X.select_dtypes(include=['object']).columns
    X = pd.get_dummies(X, columns=columnas_cualitativas)
    conversion_dict = {
      'alto': 0,
      'medio-alto': 1,
      'bajo': 2,
      'medio-bajo': 3
    }
    Y = Y.replace(conversion_dict)
    X = X.loc[:,~X.columns.duplicated()].copy()
    return X,Y

  if test == True:

    X = data.iloc[:,:11]
    columnas_cualitativas = X.select_dtypes(include=['object']).columns
    X = pd.get_dummies(X, columns=columnas_cualitativas)
    X = X.loc[:,~X.columns.duplicated()].copy()

    return X, _


In [10]:
X,Y = preproccesing('train.csv', size=0, test=False)
X.drop('ID', inplace=True, axis=1)

(692500, 21)


In [11]:
X.columns

Index(['PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL'],
      dtype='object')

In [12]:
X_train , X_test , y_train , y_test = train_test_split(X.values, Y.values, test_size = 0.2 , random_state = 42)

In [13]:
print("Train: ",X_train.shape , y_train.shape)
print("Test: ",X_test.shape , y_test.shape)

Train:  (554000, 10) (554000,)
Test:  (138500, 10) (138500,)


In [14]:
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# Definir el espacio de hiperparámetros
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'learning_rate': [0.01, 0.05, 0.1, 0.15, 0.2],
    'max_depth': [3, 4, 5, 6, 7],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.4],
    'min_child_weight': [1, 2, 3, 4, 5]
}

# Crear el modelo
xgb_model = XGBClassifier(random_state=42)

# Configurar RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=param_dist,
    n_iter=10,  # Número de combinaciones a probar
    scoring='accuracy',
    verbose=2,
    random_state=42,
    cv=3,
    n_jobs=-1  # Usar todos los núcleos disponibles
)

# Ejecutar la búsqueda
random_search.fit(X_train, y_train)

# Obtener los mejores hiperparámetros
best_params = random_search.best_params_
#print(f'Mejores hiperparámetros: {best_params}')

# Realizar predicciones con el mejor modelo
y_pred = random_search.best_estimator_.predict(X_test)

# Calcular la precisión del mejor modelo
accuracy = accuracy_score(y_test, y_pred)
print(f'Precisión: {accuracy:.4f}')

# Obtener un informe de clasificación



print(classification_report(y_test, y_pred))

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Precisión: 1.0000
              precision    recall  f1-score   support

           0       1.00      1.00      1.00    138494
           1       1.00      0.83      0.91         6

    accuracy                           1.00    138500
   macro avg       1.00      0.92      0.95    138500
weighted avg       1.00      1.00      1.00    138500



In [15]:
X,_ = preproccesing('test.csv', size=0, test=True)

(296786, 20)


In [16]:
print("Sample size: ",X.shape)

Sample size:  (296786, 11)


In [17]:
X.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL'],
      dtype='object')

In [18]:
IDs = X['ID']
X.drop('ID', inplace=True, axis=1)

In [19]:
y_pred_test = random_search.best_estimator_.predict(X.values)

In [20]:
dic = {'ID': IDs.values.T, 'RENDIMIENTO_GLOBAL': y_pred_test.T}

In [21]:
test_results = pd.DataFrame(data=dic)
test_results

,ID,RENDIMIENTO_GLOBAL
0,550236,0
1,98545,0
2,499179,0
3,782980,0
4,785185,0
...,...,...
296781,496981,0
296782,209415,0
296783,239074,0
296784,963852,0


In [22]:
conversion_dict = {
      0: 'alto',
      1: 'medio-alto',
      2: 'bajo',
      3: 'medio-bajo'
    }
test_results['RENDIMIENTO_GLOBAL'] = test_results['RENDIMIENTO_GLOBAL'].replace(conversion_dict)

In [23]:
test_results

,ID,RENDIMIENTO_GLOBAL
0,550236,alto
1,98545,alto
2,499179,alto
3,782980,alto
4,785185,alto
...,...,...
296781,496981,alto
296782,209415,alto
296783,239074,alto
296784,963852,alto


In [24]:
test_results.to_csv('modelo_XGB.csv', index=False)